In [1]:
import pandas as pd
import numpy as np 
import requests
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from sklearn.ensemble import RandomForestRegressor
import json
from datetime import datetime
import seaborn as sns

In [15]:
df_covid_limpio = pd.read_csv('df_covid_limpio.csv', sep=',')
df_covid_limpio.head()

,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,lethality_rate
0,2022-03-27,AS,American Samoa,1881,3278,2,2,0.061013
1,2022-04-03,AS,American Samoa,1378,4656,5,7,0.150344
2,2022-04-10,AS,American Samoa,746,5402,4,11,0.203628
3,2022-04-17,AS,American Samoa,133,5535,7,18,0.325203
4,2022-04-24,AS,American Samoa,230,5765,5,23,0.398959


In [17]:
df_covid_limpio["Date_reported"] = pd.to_datetime(df_covid_limpio["Date_reported"])
df_covid_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16873 entries, 0 to 16872
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date_reported      16873 non-null  datetime64[ns]
 1   Country_code       16873 non-null  object        
 2   Country            16873 non-null  object        
 3   New_cases          16873 non-null  int64         
 4   Cumulative_cases   16873 non-null  int64         
 5   New_deaths         16873 non-null  int64         
 6   Cumulative_deaths  16873 non-null  int64         
 7   lethality_rate     16873 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 1.0+ MB


In [19]:
#Creamos la función de obtener el máximo de filas
def get_max_row(group, column):
    return group.loc[group[column].idxmax()]

In [20]:
#Usando el dataframe df_covid_limpio, vamos a agrupar por país y fecha en formato (año-mes) para obtener el registro con mayor índice de letalidad en ese mes, guardamos el resultado en result.
result = df_covid_limpio.groupby(['Country', df_covid_limpio['Date_reported'].dt.to_period("M")]).apply(lambda group: get_max_row(group, 'lethality_rate'))


In [21]:
#Luego reiniciamos el índice de result.
result.reset_index(drop=True, inplace=True)

In [22]:
#Luego agrupamos el df result por nombre de país y obtenemos el indice de letalidad promedio por cada país, nos quedamos con los 4 mayores y guardamos el resultado en top_4_paises.
top_4_paises = result.groupby('Country')['lethality_rate'].mean().nlargest(4).index

In [23]:
#Luego filtramos el df result para quedarnos únicamente con los registros cuyo país está en la lista top_4_paises.
result = result[result['Country'].isin(top_4_paises)]

In [24]:
#Luego, repetimos los 2 primeros pasos pero filtrando el país que estamos analizando, guardamos el resultado en result_pais.
result_pais = df_covid_limpio[df_covid_limpio['Country'] == 'France'].groupby(df_covid_limpio['Date_reported'].dt.to_period("M")).apply(lambda group: get_max_row(group, 'lethality_rate'))
result_pais.reset_index(drop=True, inplace=True)

In [25]:
#Finalmente juntamos result y result_pais en un único dataframe result_final, para tener finalmente el top 4 países con mayor índice de letalidad y también incluir el país que estamos analizando.
result_final = pd.concat([result, result_pais])
result_final

,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,lethality_rate
331,2020-08-16,BZ,Belize,242,388,1,3,0.773196
332,2022-09-18,BZ,Belize,231,68704,0,680,0.989753
333,2022-12-25,BZ,Belize,299,69675,0,688,0.987442
334,2023-01-01,BZ,Belize,271,69946,0,688,0.983616
335,2023-02-05,BZ,Belize,50,70660,0,688,0.973677
336,2023-03-05,BZ,Belize,17,70757,0,688,0.972342
337,2023-04-02,BZ,Belize,0,70782,0,688,0.971999
338,2023-05-07,BZ,Belize,0,70782,0,688,0.971999
339,2023-06-04,BZ,Belize,7,70800,0,688,0.971751
340,2023-07-02,BZ,Belize,33,70918,0,688,0.970135
